### Pipeline

Ce notebook présente un exemple d'utilisation du pipeline implémenté dans le dossier `core/processing`. Ce code permet d'appliquer le rehaussemnet vascualaire puis la segmentation sur une image donnée avec ou sans vérité terrain et selon les paramètres de traitement sélectionnés.

In [ ]:
from core.processing.pipeline import Pipeline
from core.config.setup import SetupConfig
from core.config.experiment import ExperimentConfig, LoadingConfig, MethodsConfig, ProcessingConfig, HessianConfig, EnhancementConfig, SegmentationConfig, Experiment
from core.config.builder import ConfigBuilder

setup_config = SetupConfig(
    name="pipeline",
    input_dir="2d",
    output_dir="pipeline",
    log_file="pipline",
    debug_mode=False,
    plot_mode=False,
    save_mode=True,
)

loading_config = LoadingConfig(
    normalize=True,
    crop=False,
    target_shape=None,
    raw_file="images/image_0.png",
    gt_file="labels/label_0.png",
)

methods_config = MethodsConfig(
    derivator="default",
    enhancer='frangi',
    segmenter='thresholding',
)

processing_config = ProcessingConfig(
    use_gpu=False, # implementation not validated yet
    normalize=True, # normalization [0, 1] after enhacement
    parallelize=False, # for volumetric data, not used if 2d data
    show_progress=False, # verbose mode
    overlap_size=10, # size in voxel, not necessary if parallelize=False
    chunk_size=(128, 128, 128), # size in voxel, not necessary if parallelize=False
)

enhancement_config = EnhancementConfig(
    black_ridges=False, # vessel contrast
    scales=[2, 4, 6, 8, 10], # scale for multi-scale enhancement
    alpha=0.5, # plateness
    beta=0.5, # blobness
    gamma=None, # brightness
    skimage=False, # use frangi function from skimage, useful for checks
    hessian_function=None, # not necessary, selected from methods_config
    hessian_params=None, # not necessary, added directly in the pipeline process
)

hessian_config = HessianConfig(
    mode='reflect', # mode to deal with borders of the image
    cval=0.0, # value if mode='constant'
)

segmentation_config = SegmentationConfig(
    threshold=0.5, # threshold for binary segmentation, if None and ground_truth defined in loading_config the threshold is optimized
)

experiment_config = ExperimentConfig(
    loading=loading_config,
    methods=methods_config,
    processing=processing_config,
    hessian=hessian_config,
    enhancement=enhancement_config,
    segmentation=segmentation_config,
)

# OR

# experiment_config = ConfigBuilder(
#     config_source="pipeline.yaml",
#     config_class=ExperimentConfig,
# )

pipeline = Pipeline(setup_config)

experiment = pipeline.run(experiment_config)

data_enhanced = experiment.data_enhanced
data_segmented = experiment.data_segmented  
config = experiment.config 